# Import Necessary Libraries & Load spaCy model

HuggingFace documentation: https://github.com/huggingface/neuralcoref

In [1]:
# Import necessary libraries
import spacy
import neuralcoref
import requests
import re

# Load the model
nlp = spacy.load('en_core_web_sm')  
neuralcoref.add_to_pipe(nlp)

# Simple sentence example

In [2]:
sentence = "Corrina said she tried to pet the puppy but then it ran away from her."
sentence_doc = nlp(sentence)

for i in sentence_doc._.coref_clusters:
    print(i, "\n")

print("Resolved sentence:\n\t", sentence_doc._.coref_resolved)

Corrina: [Corrina, she, her] 

the puppy: [the puppy, it] 

Resolved sentence:
	 Corrina said Corrina tried to pet the puppy but then the puppy ran away from Corrina.


# Multiple entities example

In [3]:
mult_ent = "Alice and Bob said they like cheese, but he prefers sushi."
mult_doc = nlp(mult_ent)
mult_doc._.coref_clusters
for i in mult_doc._.coref_clusters:
    print(i, "\n")
    
print("Resolved sentence:\n\t", mult_doc._.coref_resolved)

Alice and Bob: [Alice and Bob, they] 

Bob: [Bob, he] 

Resolved sentence:
	 Alice and Bob said Alice and Bob like cheese, but Bob prefers sushi.


# Limitations: Multiple uses of the same gendered pronoun

In [4]:
mult_ent = "Corrina asked Ashley if she could pet her dog."
mult_doc = nlp(mult_ent)
mult_doc._.coref_clusters
for i in mult_doc._.coref_clusters:
    print(i, "\n")
    
print("Resolved sentence:\n\t", mult_doc._.coref_resolved)

Ashley: [Ashley, she, her] 

Resolved sentence:
	 Corrina asked Ashley if Ashley could pet Ashley dog.


# Limitations: Assumed gendered names

In [5]:
mult_ent = "Shane has a dog. She loves him."
mult_doc = nlp(mult_ent)
mult_doc._.coref_clusters
for i in mult_doc._.coref_clusters:
    print(i, "\n")
    
print("Resolved sentence:\n\t", mult_doc._.coref_resolved)

Shane: [Shane, She, him] 

Resolved sentence:
	 Shane has a dog. Shane loves Shane.


## Using the conversion dictionary parameter to help resolve words not learned in or different from training

A conversion dictionary that you can use to replace the embeddings of _rare words_ (keys) by an average of the embeddings of a list of _common words_ (values). Ex: `conv_dict={"Angela": ["woman", "girl"]}` will help resolving coreferences for `Angela` by using the embeddings for the more common `woman` and `girl` instead of the embedding of Angela. This currently only works for single words (not for words groups).

In [6]:
# Add to the conversion dictionary
nlp.remove_pipe("neuralcoref")
neuralcoref.add_to_pipe(nlp, conv_dict={'Shane': ['woman']})

# run again
doc = nlp(u'Shane has a dog. She loves him.')
for i in doc._.coref_clusters:
    print(i, "\n")
    
print("Resolved sentence:\n\t", doc._.coref_resolved)

Shane: [Shane, She] 

a dog: [a dog, him] 

Resolved sentence:
	 Shane has a dog. Shane loves a dog.


# News article sentence extraction example

[Washington Post](https://www.washingtonpost.com/dc-md-va/2023/04/11/richneck-elementary-shooting-assistant-principal-investigated/?utm_source=newsletter&utm_medium=email&utm_campaign=wp_dmv7&wpsirc=nl_dmv7&carta-url=https%3A%2F%2Fs2.washingtonpost.com%2Fcar-ln-tr%2F39b09ee%2F64368fbf53816d1ce09bf982%2F63c2ff94f7d0347081e6d512%2F12%2F52%2F64368fbf53816d1ce09bf982&wp_cu=9c21257c80b52b415e85292f2e4e6a46%7C329fd3da-b0c4-462b-8635-96bbef41c953)

In [7]:
news_article = "Authorities are exploring possible criminal charges against a former assistant principal who has been accused of ignoring warnings that a 6-year-old boy had a gun before he shot a teacher at Virginia’s Richneck Elementary School in January, according to two people familiar with the investigation. Investigators have been scrutinizing the actions of Ebony Parker, an assistant principal who resigned after the shooting, for months, the people familiar with the matter said, speaking on the condition of anonymity to discuss an ongoing investigation. The probe, though, will now probably be taken over by a special grand jury, which Newport News Commonwealth’s Attorney Howard E. Gwynn petitioned to have empaneled to keep investigating the shooting as he announced charges against the boy’s mother Monday. In his petition to impanel the special grand jury, Gwynn said it was needed to investigate “any actions or omissions” by any current or former employee of Newport News schools that contributed to the shooting, though he did not name any particular people. Parker and family members did not respond to requests for comment, and she has yet to make a public statement about the incident that left first-grade teacher Abigail Zwerner gravely wounded. It’s unclear if Parker has retained an attorney. The precise charge that Parker might face was not immediately clear. Experts said prosecuting a school administrator in connection with failures related to a shooting at a school would be highly unusual — and possibly even unprecedented — even in an era when gun violence has become a grim recurrence on campuses nationwide. The special grand jury, which was initially appointed for a six-month term, could issue additional charges in the case and make recommendations to remedy any security failures “in the hopes that such a situation never occurs again,” according to the petition made public Tuesday. Gwynn did not respond to a request for comment. The Washington Post had previously reported on allegations that school administrators downplayed warnings from staffers about the 6-year-old before the shooting. In a $40 million lawsuit filed last week, Zwerner accused Ebony Parker, along with former Richneck principal Briana Foster-Newton and former Newport News schools superintendent George Parker III, of gross negligence in failing to prevent the Jan. 6 shooting. A representative for Foster-Newton did not respond to a request for comment Tuesday, but her attorney, Pamela Branch, said previously that her client was unfairly blamed for failing to stop the shooting. “The fact of the matter is that those who were aware that the student had a gun on the premises that day did not report it to Ms. Newton,” Branch said. George Parker has declined to comment on the suit. Zwerner contends in the lawsuit that Ebony Parker was warned on at least three occasions by teachers and other school employees that the boy had a gun on the day of the shooting, and that she told the assistant principal herself the boy was in a “violent mood” and had threatened to beat up a kindergartner. In one instance, a teacher told Ebony Parker that she believed the boy may have removed a gun from his backpack and taken it outside for recess, the lawsuit alleges. The assistant principal replied that the boy had small pockets, implying he couldn’t have had a gun on him, the lawsuit alleges. Another time, a fearful student told a teacher that the boy had shown him a gun on the playground and threatened to shoot him if he told anyone, the lawsuit alleges. The teacher relayed the information to Parker through an intermediary, the suit says. Finally, less than an hour before the shooting, a guidance counselor asked Ebony Parker to search the boy for the gun after hearing a report about it, but he was rebuffed, the lawsuit alleges. About 2 p.m. on that Friday, the boy pulled out the gun and shot Zwerner as she taught in her classroom, police say. The boy brought the gun to school in his backpack; his mother had purchased it legally, police say. The mother, Deja Taylor, was charged Monday with felony child neglect and misdemeanor failing to safely store a weapon away from a child. James Ellenson, an attorney for Taylor, previously said the weapon was stored on the top shelf of a closet with a trigger lock attached. Newport News police have declined to comment on that characterization. Ellenson said Tuesday that Taylor will turn herself in by the end of the week and prosecutors have recommended a $5,000 bond. “Deja has cooperated from the first day of the incident,” Ellenson wrote in an email. “She has no criminal record.” David Riedman, the founder of the K-12 School Shooting Database, said he could not recall another high-profile school shooting case where a school administrator had been charged for allegedly failing to prevent a shooting. Prosecutors and Michigan’s attorney general said in 2021 they had not ruled out charging staffers in a shooting at Oxford High School that left four students dead, but to date no charges have been filed. Parents accused administrators of failing to heed warning signs that the shooter was dangerous. In that case, prosecutors charged the parents of the teenage shooter, Ethan Crumbley, for failing to intervene when their son showed signs of mental illness and by giving him access to the gun he used in the shootings. They have yet to stand trial; Crumbley pleaded guilty. A Broward County Sheriff’s Deputy, Scot Peterson, was charged for waiting outside Marjory Stoneman Douglas High School in Parkland, Fla., while Nikolas Cruz shot and killed 17 people inside the building in 2018. The deputy has also yet to stand trial; Cruz pleaded guilty to 17 counts each of murder and attempted murder and was sentenced to life in prison. Riedman wrote in an email it was an imperfect analogy to Richneck. “Parkland is a unique situation because a school resource officer has a very specific job description and their highest priority is stopping a shooting on campus,” Riedman wrote. “Engaging a shooter is a task that a police officer is specifically trained and equipped for.”"
# news_sentence = "Investigators have been scrutinizing the actions of Ebony Parker, an assistant principal who resigned after the shooting, for months, the people familiar with the matter said, speaking on the condition of anonymity to discuss an ongoing investigation. The probe, though, will now probably be taken over by a special grand jury, which Newport News Commonwealth’s Attorney Howard E. Gwynn petitioned to have empaneled to keep investigating the shooting as he announced charges against the boy’s mother Monday. In his petition to impanel the special grand jury, Gwynn said it was needed to investigate “any actions or omissions” by any current or former employee of Newport News schools that contributed to the shooting, though he did not name any particular people. Parker and family members did not respond to requests for comment, and she has yet to make a public statement about the incident that left first-grade teacher Abigail Zwerner gravely wounded. It’s unclear if Parker has retained an attorney."
nlp.remove_pipe("neuralcoref")
neuralcoref.add_to_pipe(nlp)
news_doc1 = nlp(news_article)

for i in news_doc1._.coref_clusters:
    print(i, "\n")

a 6-year-old boy: [a 6-year-old boy, he, the boy] 

two people familiar with the investigation: [two people familiar with the investigation, the people familiar with the matter] 

the shooting: [the shooting, the shooting, the shooting] 

Newport News Commonwealth’s Attorney Howard E. Gwynn: [Newport News Commonwealth’s Attorney Howard E. Gwynn, he, his, Gwynn, he] 

Parker: [Parker, she, Parker, Parker] 

first-grade teacher Abigail Zwerner: [first-grade teacher Abigail Zwerner, Zwerner, Zwerner] 

a shooting at a school: [a shooting at a school, the shooting, the Jan. 6 shooting, the shooting, the shooting, the shooting] 

Tuesday: [Tuesday, Tuesday] 

Ebony Parker: [Ebony Parker, George Parker, Ebony Parker, she, herself, Ebony Parker, she, Parker, Ebony Parker, she, her] 

Ms. Newton: [A representative for Foster-Newton, her, her, Ms. Newton] 

the lawsuit that Ebony Parker was warned on at least three occasions by teachers and other school employees that the boy had a gun on the d

### Grab all sentences in the article where the attorney is mentioned

In [8]:
resolved_article = news_doc1._.coref_resolved
# remove the punctuation in the entity name
resolved_article = resolved_article.replace('Newport News Commonwealth’s Attorney Howard E. Gwynn', 'Newport News Commonwealth’s Attorney Howard E Gwynn')
attorney_entity = str(news_doc1._.coref_clusters[3].main).replace(".", '')
extract_sentences = [sentence + '.' for sentence in resolved_article.split('.') if attorney_entity in sentence]
for i in extract_sentences:
    print(i, '\n')

 The probe, though, will now probably be taken over by a special grand jury, which Newport News Commonwealth’s Attorney Howard E Gwynn petitioned to have empaneled to keep investigating the shooting as Newport News Commonwealth’s Attorney Howard E Gwynn announced charges against a 6-year-old boy’s mother Monday. 

 In Newport News Commonwealth’s Attorney Howard E Gwynn petition to impanel the special grand jury, Newport News Commonwealth’s Attorney Howard E Gwynn said it was needed to investigate “any actions or omissions” by any current or former employee of Newport News schools that contributed to the shooting, though Newport News Commonwealth’s Attorney Howard E Gwynn did not name any particular people. 

